In [2]:
import pandas as pd
import numpy as np

In [3]:
# Reading CSV from link
def read_csv_from_link(url):
    path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
    df = pd.read_csv(path,delimiter="\t",error_bad_lines=False, header=None)
    return df

In [6]:
# Loading All Data
kannada_train = read_csv_from_link('https://drive.google.com/file/d/1BFYF05rx-DK9Eb5hgoIgd6EcB8zOI-zu/view?usp=sharing')
kannada_dev = read_csv_from_link('https://drive.google.com/file/d/1V077dMQvscqpUmcWTcFHqRa_vTy-bQ4H/view?usp=sharing')
kannada_test = read_csv_from_link('https://drive.google.com/file/d/14DQvnNZCXSgmiZxJqGtPYFdRqBH7TOSr/view?usp=sharing')

In [7]:
# kannada Preprocess
kannada_train = kannada_train.iloc[:, 0:2]
kannada_train = kannada_train.rename(columns={0: "text", 1: "label"})

kannada_dev = kannada_dev.iloc[:, 0:2]
kannada_dev = kannada_dev.rename(columns={0: "text", 1: "label"})

kannada_test = kannada_test.iloc[:, 0:1]
kannada_test = kannada_test.rename(columns={0: "text"})

# Stats
kannada_train['label'] = pd.Categorical(kannada_train.label)
kannada_dev['label'] = pd.Categorical(kannada_dev.label)

### Load Model

In [11]:
print("GPU Nos: {}".format(torch.cuda.device_count()))
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_name(1))

# Change Device - CPU/GPU-0/GPU-1
torch.cuda.set_device(1)
device = 'cuda'
device = device if torch.cuda.is_available() else 'cpu'

## Enter Path of Saved model here in torch.load()

In [12]:
# Model Select
import os
from os import listdir
from os.path import isfile, join
[x for x in listdir('../../finetuned_berts/') if 'kannada' in x]

['Indic_bert_offensive_only_kannada',
 'MURIL_cased_temp_offensive_only_kannada_weighted',
 'XLMroberta_large_kannada_weighted',
 'Mbert_base_cased_offensive_only_kannada',
 'Indic_bert_collated_kannada_weighted',
 'Indic_bert_kannada',
 'Distilbert_m_base_cased_kannada',
 'XLMroberta_base_kannada_weighted',
 'MURIL_cased_temp_collated_kannada_weighted',
 'XLMroberta_custom_pretrained_collated_kannada',
 'MURIL_cased_temp_collated_kannada',
 'Distilbert_m_base_cased_kannada_weighted',
 'XLMroberta_base_offensive_only_kannada_weighted',
 'XLMroberta_custom_pretrained_kannada_weighted',
 'XLMroberta_large_collated_kannada_weighted',
 'XLMroberta_base_kannada',
 'XLMroberta_custom_pretrained_offensive_only_kannada',
 'Indic_bert_collated_kannada',
 'MURIL_cased_temp_kannada_weighted',
 'XLMroberta_large_kannada',
 'Mbert_base_cased_collated_kannada_weighted',
 'XLMroberta_custom_pretrained_collated_kannada_weighted',
 'XLMroberta_custom_pretrained_offensive_only_kannada_weighted',
 'Mbert

In [220]:
models = []
tokenizers = []

In [221]:
from transformers import AutoTokenizer, AutoModel, BertTokenizer, XLMRobertaTokenizer

model_names = [
    'XLMroberta_base_kannada',
    'XLMroberta_custom_pretrained_kannada',
    'Mbert_base_cased_kannada',
]
tokenizers = [
    XLMRobertaTokenizer.from_pretrained('xlm-roberta-base'),
    XLMRobertaTokenizer.from_pretrained('xlm-roberta-base'),
    BertTokenizer.from_pretrained('bert-base-multilingual-cased'),
]

for name in model_names:
    model = AutoModel.from_pretrained(os.path.join('../../finetuned_berts/', name))
    model.eval()
    models.append(model)

Some weights of XLMRobertaModel were not initialized from the model checkpoint at ../../finetuned_berts/XLMroberta_base_kannada and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of XLMRobertaModel were not initialized from the model checkpoint at ../../finetuned_berts/XLMroberta_custom_pretrained_kannada and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [222]:
lin_dim = np.sum([1024 if 'large' in name else 768 for name in model_names])
lin_dim

2304

In [223]:
n_models = len(models)

In [224]:
for model in models:
    for param in model.parameters():
        param.requires_grad = False

### Dataset Stuff

In [226]:
label_mapping = {
    'Not_offensive': 0, 
    'not-Kannada': 1, 
    'Offensive_Targeted_Insult_Other': 2, 
    'Offensive_Targeted_Insult_Group': 3, 
    'Offensive_Untargetede': 4, 
    'Offensive_Targeted_Insult_Individual': 5
}

# Collecting Text and Labels
train_batch_sentences = list(kannada_train['text'])
train_batch_labels =  [label_mapping[x] for x in kannada_train['label']]
dev_batch_sentences = list(kannada_dev['text'])
dev_batch_labels =  [label_mapping[x] for x in kannada_dev['label']]
test_batch_sentences = list(kannada_test['text'])

# Convert to Tensor
train_encodings = [tokenizer(train_batch_sentences, padding='max_length', truncation=True, max_length=64, return_tensors="pt") for tokenizer in tokenizers]
train_labels = torch.tensor(train_batch_labels)
dev_encodings = [tokenizer(dev_batch_sentences, padding='max_length', truncation=True, max_length=64, return_tensors="pt") for tokenizer in tokenizers]
dev_labels = torch.tensor(dev_batch_labels)
test_encodings = [tokenizer(test_batch_sentences, padding='max_length', truncation=True, max_length=64, return_tensors="pt") for tokenizer in tokenizers]

In [227]:
# Dataset
from torch.utils.data import Dataset

class kannada_Offensive_Dataset(Dataset):
    def __init__(self, encodings, labels = None):
        self.encodings = encodings
        self.labels = labels
        self.n_models = len(encodings)

    def __getitem__(self, idx):
        item = {}
        for i in range(self.n_models):
            item.update({key+'_'+str(i): torch.tensor(val[idx]) for key, val in self.encodings[i].items()})
        item['index'] = idx
        if self.labels != None:
            item['labels'] = torch.tensor(self.labels[idx])
        else:
            item['labels'] = torch.tensor(1)
        return item

    def __len__(self):
        return len(self.encodings[0]['input_ids'])

# Defining Datasets
train_dataset = kannada_Offensive_Dataset(train_encodings, train_labels)
dev_dataset = kannada_Offensive_Dataset(dev_encodings, dev_labels)
test_dataset = kannada_Offensive_Dataset(test_encodings)

#### FUSION MODEL

In [228]:
import torch.nn.functional as F
import torch.nn as nn

# Basic Fully-Connected (Linear => BatchNorm => ReLU)
class BasicFC(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicFC, self).__init__()
        self.fc = nn.Linear(in_channels, out_channels, **kwargs)
        self.bn = nn.BatchNorm1d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.fc(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

class FusionNet(torch.nn.Module):
    def __init__(self, D_in, H1, H2, H3, D_out):
        super(FusionNet, self).__init__()
        self.linear1_1 = BasicFC(D_in, H1)
        self.linear1_2 = BasicFC(H1, H2)
        self.linear1_3 = BasicFC(H2, H3)
        self.dp = nn.Dropout(0.1)
        self.linear2 = torch.nn.Linear(H3, D_out, bias = False)

    def forward(self, x):
        h_relu_1 = self.linear1_1(x)
        h_relu_2 = self.dp(self.linear1_2(h_relu_1))
        h_relu_3 = self.dp(self.linear1_3(h_relu_2))
        y_pred = self.linear2(h_relu_3)
        return y_pred

In [229]:
# Optimiser
from transformers import AdamW

In [230]:
## Loss Fn
XE_loss_function = nn.CrossEntropyLoss(reduction='mean').float()

In [231]:
class F1_Loss(nn.Module):
    '''Calculate F1 score. Can work with gpu tensors
    
    The original implmentation is written by Michal Haltuf on Kaggle.
    
    Returns
    -------
    torch.Tensor
        `ndim` == 1. epsilon <= val <= 1
    
    Reference
    ---------
    - https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
    - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    - https://discuss.pytorch.org/t/calculating-precision-recall-and-f1-score-in-case-of-multi-label-classification/28265/6
    - http://www.ryanzhang.info/python/writing-your-own-loss-function-module-for-pytorch/
    '''
    def __init__(self, epsilon=1e-7, n_labels = 6):
        super().__init__()
        self.epsilon = epsilon
        self.n_labels = n_labels
        
    def forward(self, y_pred, y_true,):
        assert y_pred.ndim == 2
        assert y_true.ndim == 1
        y_true = F.one_hot(y_true, self.n_labels).to(torch.float32)
        y_pred = F.softmax(y_pred, dim=1)
        
        tp = (y_true * y_pred).sum(dim=0).to(torch.float32)
        tn = ((1 - y_true) * (1 - y_pred)).sum(dim=0).to(torch.float32)
        fp = ((1 - y_true) * y_pred).sum(dim=0).to(torch.float32)
        fn = (y_true * (1 - y_pred)).sum(dim=0).to(torch.float32)

        precision = tp / (tp + fp + self.epsilon)
        recall = tp / (tp + fn + self.epsilon)

        f1 = 2* (precision*recall) / (precision + recall + self.epsilon)
        f1 = f1.clamp(min=self.epsilon, max=1-self.epsilon)
        return 1 - f1.mean()

F1_loss_function = F1_Loss().cuda()

In [232]:
use_f1_loss = False
if use_f1_loss:
    loss_function = F1_loss_function
else:
    loss_function = XE_loss_function

In [233]:
add_extra_embeds = [
    'cnn_128',
    #'sentbert',
    #'laser_ta',
    #'laser_en',
]

embeds_files = {
    'cnn_128': ['../../sentence_embeddings/cnn_emb_train_128_kannada.npy', '../../sentence_embeddings/cnn_emb_dev_128_kannada.npy', '../../sentence_embeddings/cnn_emb_test_128_kannada.npy'],
    'laser_kn': ['../../sentence_embeddings/LASER_embeddings_kannada_kn_train.npy', '../../sentence_embeddings/LASER_embeddings_kannada_kn_dev.npy', '../../sentence_embeddings/LASER_embeddings_kannada_kn_test.npy'],
    'laser_en': ['../../sentence_embeddings/LASER_embeddings_kannada_en_train.npy', '../../sentence_embeddings/LASER_embeddings_kannada_en_dev.npy', '../../sentence_embeddings/LASER_embeddings_kannada_en_test.npy'],
    'sentbert': ['../../sentence_embeddings/sentence_bert_kannada_train.npy', '../../sentence_embeddings/sentence_bert_kannada_dev.npy'],
}

train_embeddings = [np.load(embeds_files[embname][0]) for embname in add_extra_embeds]
dev_embeddings = [np.load(embeds_files[embname][1]) for embname in add_extra_embeds]
test_embeddings = [np.load(embeds_files[embname][2]) for embname in add_extra_embeds]

In [234]:
len_extra_embeds = len(add_extra_embeds)
len_extra_embeds

1

In [235]:
dim_extra_embeds = np.sum([train_embeddings[i].shape[1] for i in range(len_extra_embeds)])
dim_extra_embeds

128

In [236]:
print(lin_dim + dim_extra_embeds)

2432


In [238]:
print(n_models)

3


In [239]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, f1_score

fusion_classifier = FusionNet(int(lin_dim + dim_extra_embeds), 1024, 256, 64, 6)
# Optimiser
optimizer = AdamW(fusion_classifier.parameters(), lr=1e-6)
mo,ex_em = "",""
for mod in model_names:
    mo += mod
for em in add_extra_embeds:
    ex_em += em
model_name = 'fusion_kannada_'+mo+'_'+ex_em

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
fusion_classifier.to(device)
for model in models:
    model.to(device)
best_val_f1 = 0
count = 0

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
dev_loader = DataLoader(dev_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
PATH = ""
for epoch in range(100):
    fusion_classifier.train()
    print("==========================================================")
    print("Epoch {}".format(epoch))
    print("Train")
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        outputs_all = []
        for i in range(n_models):
            model = models[i]
            input_ids = batch['input_ids'+'_'+str(i)].to(device)
            attention_mask = batch['attention_mask'+'_'+str(i)].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            outputs_all.append(outputs[1])
        
        for i in range(len_extra_embeds):
            outputs_all.append(torch.Tensor(train_embeddings[i][batch['index'], :]).to(device))
            
        bert_output = torch.cat(outputs_all, dim = -1) 
        out = fusion_classifier(bert_output)
        loss = loss_function(out, labels)
        loss.backward()
        optimizer.step()
    
    print("Dev")
    dev_preds,dev_probs = [],[]
    fusion_classifier.eval()
    total_val_loss = 0
    with torch.set_grad_enabled(False):
        for batch in tqdm(dev_loader):
            outputs_all = []
            for i in range(n_models):
                model = models[i]
                input_ids = batch['input_ids'+'_'+str(i)].to(device)
                attention_mask = batch['attention_mask'+'_'+str(i)].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids, attention_mask=attention_mask)
                outputs_all.append(outputs[1])
                
            for i in range(len_extra_embeds):
                outputs_all.append(torch.Tensor(dev_embeddings[i][batch['index'], :]).to(device))

            bert_output = torch.cat(outputs_all, dim = -1) 
            out = fusion_classifier(bert_output)
            loss = loss_function(out, labels)
            total_val_loss += loss.item()/len(dev_loader)
            
            for logits in out.cpu().numpy():
                dev_preds.append(np.argmax(logits))
            for logits in out.cpu().numpy():
                dev_probs.append(np.exp(logits)/np.sum(np.exp(logits)))
    
    y_true = dev_batch_labels
    y_pred = dev_preds
    target_names = label_mapping.keys()
    report = classification_report(y_true, y_pred, target_names=target_names)
    val_f1 = f1_score(y_true, y_pred, average='macro')
    
    if val_f1 > best_val_f1:
        PATH = '../../finetuned_models/' + model_name + '.pth'
        torch.save(fusion_classifier.state_dict(), PATH)
        
        #save dev preds
        with open('dev_preds/'+str(model_name)+'.npy', 'wb') as f:
            print(np.array(dev_probs).shape)
            np.save(f, np.array(dev_probs))
            
        #find test preds
        print("Test")
        test_preds,test_probs = [],[]
        fusion_classifier.eval()
        total_test_loss = 0
        with torch.set_grad_enabled(False):
            for batch in tqdm(test_loader):
                outputs_all = []
                for i in range(n_models):
                    model = models[i]
                    input_ids = batch['input_ids'+'_'+str(i)].to(device)
                    attention_mask = batch['attention_mask'+'_'+str(i)].to(device)
                    labels = batch['labels'].to(device)
                    outputs = model(input_ids, attention_mask=attention_mask)
                    outputs_all.append(outputs[1])

                for i in range(len_extra_embeds):
                    outputs_all.append(torch.Tensor(test_embeddings[i][batch['index'], :]).to(device))

                bert_output = torch.cat(outputs_all, dim = -1) 
                out = fusion_classifier(bert_output)
                loss = loss_function(out, labels)
                total_test_loss += loss.item()/len(test_loader)

                for logits in out.cpu().numpy():
                    test_preds.append(np.argmax(logits))
                for logits in out.cpu().numpy():
                    test_probs.append(np.exp(logits)/np.sum(np.exp(logits)))

        y_pred = test_preds
        target_names = label_mapping.keys()
        with open('test_preds/'+str(model_name)+'.npy', 'wb') as f:
            print(np.array(test_probs).shape)
            np.save(f, np.array(test_probs))
            
        best_val_f1 = val_f1
        count = 0
    else:
        count += 1
    
    print(report)
    print("Epoch {}, Val Loss = {}, Val F1 = {}, Best Val f1 = {}, stagnant_t = {}".format(epoch, total_val_loss, val_f1, best_val_f1, count))
    if count == 5:
        print("No increase for 5 epochs, Stopping ...")
        break

Epoch 0
Train


/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/punyajoy/.conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()



Dev



(777, 6)
Test



(778, 6)
                                      precision    recall  f1-score   support

                       Not_offensive       0.81      0.78      0.80       426
                         not-Kannada       0.75      0.81      0.78       191
     Offensive_Targeted_Insult_Other       0.14      0.25      0.18        16
     Offensive_Targeted_Insult_Group       0.33      0.38      0.35        45
               Offensive_Untargetede       0.14      0.03      0.05        33
Offensive_Targeted_Insult_Individual       0.51      0.55      0.53        66

                            accuracy                           0.70       777
                           macro avg       0.45      0.47      0.45       777
                        weighted avg       0.70      0.70      0.70       777

Epoch 0, Val Loss = 1.3049181602439104, Val F1 = 0.44797332147558144, Best Val f1 = 0.44797332147558144, stagnant_t = 0
Epoch 1
Train


...


                                      precision    recall  f1-score   support

                       Not_offensive       0.78      0.82      0.80       426
                         not-Kannada       0.75      0.81      0.78       191
     Offensive_Targeted_Insult_Other       0.17      0.06      0.09        16
     Offensive_Targeted_Insult_Group       0.38      0.36      0.37        45
               Offensive_Untargetede       0.29      0.15      0.20        33
Offensive_Targeted_Insult_Individual       0.64      0.56      0.60        66

                            accuracy                           0.72       777
                           macro avg       0.50      0.46      0.47       777
                        weighted avg       0.70      0.72      0.71       777

Epoch 8, Val Loss = 1.0706197230183345, Val F1 = 0.4716575030830532, Best Val f1 = 0.4865329847370578, stagnant_t = 3
Epoch 9
Train



Dev



                                      precision    recall  f1-score   support

                       Not_offensive       0.80      0.77      0.79       426
                         not-Kannada       0.72      0.85      0.78       191
     Offensive_Targeted_Insult_Other       0.10      0.06      0.08        16
     Offensive_Targeted_Insult_Group       0.38      0.44      0.41        45
               Offensive_Untargetede       0.19      0.12      0.15        33
Offensive_Targeted_Insult_Individual       0.65      0.56      0.60        66

                            accuracy                           0.71       777
                           macro avg       0.47      0.47      0.47       777
                        weighted avg       0.71      0.71      0.71       777

Epoch 9, Val Loss = 1.1102136069414568, Val F1 = 0.46822890360625974, Best Val f1 = 0.4865329847370578, stagnant_t = 4
Epoch 10
Train



Dev



                                      precision    recall  f1-score   support

                       Not_offensive       0.81      0.77      0.79       426
                         not-Kannada       0.73      0.83      0.78       191
     Offensive_Targeted_Insult_Other       0.12      0.12      0.12        16
     Offensive_Targeted_Insult_Group       0.35      0.40      0.38        45
               Offensive_Untargetede       0.24      0.18      0.21        33
Offensive_Targeted_Insult_Individual       0.61      0.56      0.58        66

                            accuracy                           0.71       777
                           macro avg       0.48      0.48      0.48       777
                        weighted avg       0.71      0.71      0.71       777

Epoch 10, Val Loss = 1.126094503062112, Val F1 = 0.4759235613349075, Best Val f1 = 0.4865329847370578, stagnant_t = 5
No increase for 5 epochs, Stopping ...


In [240]:
print(model_name)

fusion_kannada_XLMroberta_base_kannadaXLMroberta_custom_pretrained_kannadaMbert_base_cased_kannada_cnn_128


In [241]:
file = open("test_preds/fusion_kannada_XLMroberta_base_kannadaIndic_bert_kannada_.npy",'rb')
object_file = np.load(file)
print(object_file.shape)

(778, 6)


#### Save config in dict for predictions and ensemble

In [188]:
dict = {}
dict['pth_name'] = model_name
dict['model'] = model_names
dict['pretrained_keys'] = ['xlm-roberta-base','xlm-roberta-base','bert-base-multilingual-cased']
dict['extra_embeds'] = add_extra_embeds
dict['length'] = lin_dim + dim_extra_embeds

In [190]:
model_name_tot = 'dicts/'+model_name

In [191]:
import pickle
with open(str(model_name_tot)+'.pickle', 'wb') as handle:
    pickle.dump(dict, handle)

### Save Preds - Get predictions from individual model

In [ ]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report, f1_score

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Dataloaders
dev_loader = DataLoader(dev_dataset, batch_size=16, shuffle=False)
fusion_classifier.load_state_dict(torch.load('../../finetuned_models/' + model_name + '.pth'))
fusion_classifier.eval()

dev_preds = []
with torch.set_grad_enabled(False):
    for batch in tqdm(dev_loader):
        outputs_all = []
        for i in range(n_models):
            model = models[i]
            input_ids = batch['input_ids'+'_'+str(i)].to(device)
            attention_mask = batch['attention_mask'+'_'+str(i)].to(device)
            labels = batch['labels'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            outputs_all.append(outputs[1])
            
        for i in range(len_extra_embeds):
            outputs_all.append(torch.Tensor(dev_embeddings[i][batch['index'], :]).to(device))

        bert_output = torch.cat(outputs_all, dim = -1) 
        out = fusion_classifier(bert_output)
            
        for logits in out.cpu().numpy():
            dev_preds.append(np.argmax(logits))

In [ ]:
y_true = dev_batch_labels
y_pred = dev_preds
target_names = label_mapping.keys()
report = classification_report(y_true, y_pred, target_names=target_names)

In [ ]:
print(report)

In [ ]:
np.savetxt("../../dev_preds/" + model_name[:-4] + ".csv", dev_preds, delimiter=",")